Imports for Numpy and Computer Vision

In [ ]:
import cv2
import numpy as np

In [41]:
from google.colab import drive
root = '/content/drive/'
drive.mount(root)

import os
from os.path import join
os.chdir('/content/drive/My Drive/Colab Notebooks')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Get grayscale of two images and then take red components

Then crop images to become a square

In [42]:
image1 = cv2.imread("birdImage1.png")[:,:,0] # take red components
image2 = cv2.imread("birdImage2.png")[:,:,0] # take red components
crop_size = 1800
image1_cp = image1[:crop_size, :crop_size] # crop to square
image2_cp = image2[:crop_size, :crop_size] # crop to square

Use the OpenCV DFT function on image, and then shift the image so that the origin of the coordinate comes at the image center

In [43]:
f1 = cv2.dft(image1_cp.astype(np.float32),
              flags=cv2.DFT_COMPLEX_OUTPUT)
f2 = cv2.dft(image2_cp.astype(np.float32),
              flags=cv2.DFT_COMPLEX_OUTPUT)
f1_shf = np.fft.fftshift(f1)
f2_shf = np.fft.fftshift(f2)

Then, get the complex value from the shift values

In [44]:
f1_shf_cplx = f1_shf[:,:,0] + 1j*f1_shf[:,:,1]
f2_shf_cplx = f2_shf[:,:,0] + 1j*f2_shf[:,:,1]

Then, use the shift's absolute values in order to evaluate the P complex , which is the product of two complex numbers.

P_complex represents the value/equation as a whole for the complex number value.

In [45]:
f1_shf_abs = np.abs(f1_shf_cplx)
f2_shf_abs = np.abs(f2_shf_cplx)
total_abs = f1_shf_abs * f2_shf_abs
P_real = (np.real(f1_shf_cplx)*np.real(f2_shf_cplx) +
          np.imag(f1_shf_cplx)*np.imag(f2_shf_cplx))/total_abs
P_imag = (np.imag(f1_shf_cplx)*np.real(f2_shf_cplx) +
          np.real(f1_shf_cplx)*np.imag(f2_shf_cplx))/total_abs
P_complex = P_real + 1j*P_imag

Calculate the inverse Fourier transform for P_complex, which will result a real image that has mostly back pixels with one shining white pixel.

This tells you how much the frames have shifted

In [46]:
P_inverse = np.abs(np.fft.ifft2(P_complex)) # inverse FFT
max_id = [0, 0]
max_val = 0

# Get the shape of P_inverse to avoid hardcoding the dimensions
rows, cols = P_inverse.shape

# Iterate using the actual shape of P_inverse
for idy in range(rows):
    for idx in range(cols):
        if P_inverse[idy,idx] >= max_val:
            max_val = P_inverse[idy,idx]
            max_id = [idy, idx]

shift_x = crop_size - max_id[0]
shift_y = crop_size - max_id[1]
print(shift_x, shift_y)

1800 1624


Then you the shift to visualize the shift in a way that images within the recantagle coincide each other

In [47]:
image1_rgb = cv2.imread("birdImage1.png")
image2_rgb = cv2.imread("birdImage2.png")
cv2.rectangle(image1_rgb, (0, 0), (image1_rgb.shape[0] - shift_x, image1_rgb.shape[1] - shift_y), (0, 255, 0), 10)
cv2.rectangle(image2_rgb, (shift_x, shift_y), (image2_rgb.shape[0], image2_rgb.shape[1]), (0, 255, 0), 10)
cv2.imwrite("image1_rectangle.jpg", image1_rgb)
cv2.imwrite("image2_rectangle.jpg", image2_rgb)

True